In [230]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
import os 
import scipy

In [231]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *

1.13.1+cu117


In [232]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

from scipy.stats import wilcoxon
import anndata
import scanpy as sc

# Pre-processing from original matrix


In [91]:
# matrix_list = []
# common_genes = []
# for i,j in zip(range(3,8),range(5,10)):
#     matrix_list.append(pd.read_csv(f"/data/raw_data/Drosophila/GSM249478{j}_dge_mel_rep{i}.txt",sep="\t",index_col=0))
#     if not common_genes:
#         common_genes = pd.read_csv(f"/data/raw_data/Drosophila/GSM249478{j}_dge_mel_rep{i}.txt",sep="\t",index_col=0).index.tolist()
#     else:
#         common_genes = list(set(common_genes) & set(pd.read_csv(f"/data/raw_data/Drosophila/GSM249478{j}_dge_mel_rep{i}.txt",sep="\t",index_col=0).index.tolist()))
#         #matrix_list = [i.loc[common_genes] for i in matrix_list]
# matrix = pd.concat(matrix_list)


# # In[ ]:


# matrix = matrix.loc[common_genes]
# matrix = matrix[~matrix.index.duplicated(keep='first')]


# # In[ ]:


# matrix.index = [str(i).upper() for i in matrix.index.tolist()]

In [183]:
#matrix = pd.read_csv("../data/GSE95025/GSM2494785_dge_mel_rep3.txt",sep="\t")
matrix = pd.read_csv("/data/raw_data/Pre_Post/GSE150949/pc9_t7_rep2.csv",index_col=0)

In [184]:
index = matrix.index.tolist()

In [185]:
matrix = matrix.fillna(0)

In [186]:
adata = anndata.AnnData(matrix.transpose())

In [187]:
sc.pp.filter_genes(adata, min_cells=10)

In [188]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)

         Falling back to preprocessing with `sc.pp.pca` and default params.


In [189]:
meta = pd.DataFrame({"cell":adata.obs["leiden"].index.tolist(),"labels":adata.obs["leiden"].tolist()})

In [190]:
meta.index = meta["cell"].tolist()

In [191]:
adata.obs = meta

In [192]:
matrix = pd.DataFrame(adata.X.transpose(),columns=adata.obs.index.tolist(),index=adata.var.index.tolist())

# preprocessing with h5ad

In [233]:

adata = sc.read_h5ad("/data/raw_data/Cardiac_cells/Visium-FZ_GT_P19.h5ad")
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)
matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
meta = pd.DataFrame({"cell":adata.obs.index.tolist(),"labels":adata.obs["cell_type_original"].tolist()})
meta.index = meta["cell"].tolist()
#removing lymphoid as it is only one cell, cannot be used in benchmarking
meta = meta[meta['labels'] != "Lymphoid"]
adata = adata[meta["cell"].tolist()]
matrix = matrix.transpose()

matrix = matrix[meta.index.tolist()]

index = matrix.index.tolist()
adata.obs = meta


# rest of preprocessing

In [234]:
#sc.tl.rank_genes_groups(adata, 'labels')
cell_groups = meta['labels'].unique().tolist()
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    temp_matrix = matrix[cells]
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) > 0].index.tolist())

In [235]:
adata = adata.transpose()

In [236]:
#adata.obs["Gene"] = matrix.index.astype("category")

In [237]:
# pval_df = pd.DataFrame.from_records(adata.uns["rank_genes_groups"]["pvals"])
# pval_df.index = adata.obs.index.tolist()

In [238]:
# cell_type_df = {}
# for i in pval_df.columns.tolist():
#     sub = pval_df[i]
#     sub = sub[sub < 0.05]
#     cell_type_df[i] = sub.index.tolist()

In [239]:
cell_type_df = matrix_list

In [240]:
#mean_expression = np.mean(matrix.values[matrix.values != 0])
nodes = pd.DataFrame({"category":[],"identifier":[]})

In [241]:
LR_nodes = pd.read_csv("/data/LR_database/OmniPath_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("/data/LR_database/OmniPath_interactions.csv",index_col=0)

In [242]:
ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

In [243]:
ligand_list = []
receptor_list = []
new_cell_df = {}
for i in cell_type_df.keys():
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
    new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]

In [244]:
for i in new_cell_df.keys():
    new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
    new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]

In [245]:
ligand_list = list(set(ligand_list))
receptor_list = list(set(receptor_list))

In [246]:
ligand_list = [i+"_Ligand" for i in ligand_list]
receptor_list = [i+"_Receptor" for i in receptor_list]

In [247]:
nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])

In [248]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [249]:
LR_nodes["identifier"] = new_identifier

In [250]:
nodes["Id"] = range(0,nodes.shape[0])
nodes = nodes[["Id","category","identifier"]]

In [251]:
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

In [252]:
meta.index = meta["cell"].tolist()

In [253]:
interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})

In [254]:
LR_nodes.index = LR_nodes["Id"].tolist()

In [255]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


In [256]:
source_list = []
dest_list = []
weight_list = []
edge_type_list = []
for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    dest_list.extend(new_cell_df[i][0])
    dest_list.extend(new_cell_df[i][1])
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))

In [257]:
interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

In [258]:
nodes.index = nodes["identifier"].tolist()

In [259]:
nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0,nodes.shape[0])

In [260]:
interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

In [261]:
first_data,first_nodes,first_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)

# Edit the ground truth to only contain information from this dataset

In [262]:
LR_nodes.index = LR_nodes["identifier"].tolist()

In [263]:
#LR_nodes["identifier"] = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [264]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [265]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()

In [266]:
LR_nodes["Id"] = range(0,LR_nodes.shape[0])
LR_nodes.index = LR_nodes["identifier"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [267]:
nodes.to_csv("/data/GraphComm_Input/Cardiac_cells/nodes.csv")
interactions.to_csv("/data/GraphComm_Input/Cardiac_cells/interactions.csv")
meta.to_csv("/data/GraphComm_Input/Cardiac_cells/meta.csv")
matrix.to_csv("/data/GraphComm_Input/Cardiac_cells/matrix.csv")
LR_nodes.to_csv("/data/GraphComm_Input/Cardiac_cells/LR_nodes.csv")
Omnipath_network.to_csv("/data/GraphComm_Input/Cardiac_cells/Omnipath_network.csv")